## Import

# Houseing Prices

## Content Table

// content table

## Information

La target est la colonne `Sale Price` qui donne le prix de vente d'une maison.
Plus d'informations sur les valeurs et la signification des colonnes est disponible dans le fichier `data_description.txt`

Pré: Importation des packages de bases

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV


Post: Les packages sont importés

## Load data

Pré: Chargement des données dans un dataframe depuis le fichier `train.csv`

In [ ]:
housing_prices_all = pd.read_csv('./train.csv',)
housing_prices_all.head()

Post: Le dataframe est chargé dans la variable `housing_prices`

Pré: Suppression de la colonne Id qui n'apporte aucune information

In [ ]:
housing_prices_all = housing_prices_all.drop('Id', axis=1)

Post: La colonne Id a été supprimé

#### Features Selection

Pré: Affichage d'un graphique heatmap des correlations entre les caractéristiques.

In [ ]:
plt.figure(figsize=(12, 8))
plt.title("Heatmap correlation")
sns.heatmap(housing_prices_all.corr())

Post:La dernière ligne est celle qui nous intéresse le plus, elle montre la corrélation entre le prix de vente et les caractéristiques.

Nous pouvons remarquer que les caractéristiques les plus importantes sont `OverallQual` et `GrLiveArea`, suivi par `YearBuilt`, `YearRemodAdd` et `TotRmsAbvGrd`.

Par contre, des caractéristiques comme `MSSubClass`, `OverallCond` ou `KitchenAbvGr` ont très peu d'influence sur le prix d'une maison.

Pré: Sélection de 10/15 colonnes de façon arbitraire pour alléger la charge de travail.

In [ ]:
categorical_columns = ['OverallQual', 'MSZoning', 'LandContour',
                       'Utilities', 'Neighborhood', 'BldgType', 'Exterior1st', 'GarageType']
numerical_columns = ['LotFrontage', 'LotArea',
                     'YearBuilt', 'BedroomAbvGr', 'GrLivArea', 'TotRmsAbvGrd']

housing_prices = housing_prices_all[categorical_columns + numerical_columns + ['SalePrice']]


Post: Explications des features choisies.

Categoriels:
  - OverallQual: Feature obligatoire + elle assez lié au prix d'une maison.
  - MSZoning: décrit la zone où se trouve la maison. Semble une caractéristique intéressante
  - LandContour: décrit à quel point le terrain est plat.
  - Utilities: décrit s'il y a de l'eau, gaz, électricité, etc
  - Neighborhood: Le quartier dans lequel se situe la maison
  - BldgType: Le type de batiment
  - Exterior1st: le matériau principal de la maison
  - GarageType: si la maison possède un garage et de quel type

Numerique:
  - LotFrontage: le nombre de pied de la propriété relié à la route.
  - LotArea: La taille de la propriété en pieds carrés
  - YearBuilt: L'année de construction
  - GrLiveArea: la surface habitable au dessus du sol (sans la cave)
  - BedroomAbvGr: le nombre de chambre
  - TotRmdAbvGrd: Le nombre de pièces au dessus du sol sans les salles de bains.

La majorité des features choisies sont en fonction des caractéristiques qui semblent les plus importantes dans le choix d'une maison et qui donc vont peut-être influencé le prix.

Nous avons donc un DataFrame avec uniquement les colonnes que l'on désire analyser avec en plus la colonne `SalePrice` qui est notre target


Pré: affichage des informations du dataframe

In [ ]:
housing_prices.info()

Post: Les informations du dataframe sont affichées. Nous pouvons voir le type de chaque colonnes ainsi que les colonnes comprenant des éléments `null`

Pré:  Affichage des colonnes ayant des éléments `null`
Les colonnes comme `LotFrontage` devrait apparaitre

In [ ]:
columns_null_values = housing_prices.isnull().sum(axis=0)
columns_null_values = columns_null_values[columns_null_values > 0]
print(columns_null_values.shape[0])
columns_null_values

Post: Il y a 2 colonnes comprenants des valeurs nulls
- GarageType: est à `None` si il n'y a pas de garage. Peut être remplacé par la chaine de caractère `"None"`
- LotFrontage: Si il n'y a pas de valeurs, c'est que la maison n'est pas relié à la route directement. Peut donc être remplacé par `0`
  
Ces valeurs seront remplacer grace à  des `SimpleImputer` dans le pipeline.

## Data Analyse

Pré: Affichage de la comparaison entre toutes les features numériques entre-elles.

In [ ]:
sns.pairplot(housing_prices[numerical_columns + ['SalePrice']],)


Post: Nous pouvons remarquer qu'il y a quelques valeurs extrêmes dans `LotFrontage` et `LotArea`. Le prix d'une maisons semble assez lié à des features comme`GrLivArea` et `TotRmdAbvGrd`  

Pré: Comparaison du prix d'une maison par rapport à sa qualité de finition.

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title("Prix moyen d'une maison par rapport à sa qualité de finitions")
sns.lineplot(x="OverallQual", y="SalePrice", data=housing_prices)
plt.subplot(1, 2, 2)
plt.title("Répartition des prix par qualité de finitions")
sns.boxplot(x="OverallQual", y="SalePrice", data=housing_prices)


Post: Au plus la qualité augmente, au plus le prix d'une maison augmente. Mais au plus une maison est de bonne qualité, au plus les prix varient. 

Pré: Affichage de l'évolution du prix d'une maison par rapport à la surface habitable au dessus du sol avec sa regression lineair.

In [ ]:
plt.title("Evolution du prix d'une maison par rapport à la surface habitable au dessus du sol")
sns.lineplot(x="GrLivArea", y="SalePrice", data=housing_prices)
sns.regplot(x="GrLivArea", y="SalePrice", data=housing_prices, scatter=False)


Post: Nous pouvons voir qu'il y a un lien entre les 2 caractéristiques jusqu'à environ 3500 pieds carrés. Après cette valeurs, le prix des maisons ne semble plus être lié avec la surface.

Pré: Affichage du prix d'une maison par rapport à l'année de construction

In [ ]:
plt.title("Evolution du prix d'une maison par rapport à l'année de construction")
sns.lineplot(x="YearBuilt", y="SalePrice", data=housing_prices)
sns.regplot(x="YearBuilt", y="SalePrice", data=housing_prices, scatter=False)


Post: L'année de construction influe le prix d'une maison

Pré: Affichage de l'évolution du prix d'une maison par rapport au total de pieces au dessus du sol.

In [ ]:
plt.title("Evolution du prix d'une maison par rapport au total de pieces au dessus du sol.")
sns.lineplot(x="TotRmsAbvGrd", y="SalePrice", data=housing_prices)


Post: Le prix d'une maison semble être lié au nombre de pièces dans une maison. Sauf si le nombre de pièce dépasse 12.

Pré: Affichage de la répartition du prix des maisons

In [ ]:
sns.displot(x="SalePrice", data=housing_prices)
plt.title("Répartition du prix des maisons")

Post: La répartition du prix des maisons a été affichée.

## Data Modeling

Pré: Séparation des features et de la target

In [ ]:
X = housing_prices_all.drop('SalePrice', axis=1)
y = housing_prices_all['SalePrice']

Post: X contient les Features et y la target pour chaque ligne

Pré: Séparation du train et test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Post: Le train et test set ont été séparés. Il y a 20% des données dans le test set.

Pré: Création des pipelines pour le pré-processing

In [ ]:
numerical_pipeline = make_pipeline(SimpleImputer(strategy="constant", fill_value=0),StandardScaler())
categorical_pipeline = make_pipeline(SimpleImputer(strategy="constant", fill_value="None"), OneHotEncoder(handle_unknown="ignore"))

preprocessor = make_column_transformer(
    (numerical_pipeline, numerical_columns), (categorical_pipeline, categorical_columns))


Post: `numerical_pipeline` va transformer toutes les valeurs nulls par des `0` et `categorical_pipeline` par des `None`. Ensuiite les valeurs numériques vont être transformer grâce au `StandardScaler` et les catégories grâce au `OneHotEncoder` 

Un preprocessor est donc créé grâce aux 2 pipelines. Il sera utilisé par la suite dans la création du modèle.

### Linear Regression
pour un problème de ce type, il faut utiliser la régression. On ne cherche pas à catégoriser un ensemble de donnés mais à trouver une "quantité", le prix d'une maison.
Nous allons donc uniquement utilisé la regression linéaire

Pré: Utilisation de GridSearchCV avec une pipeline qui contient le preprocessor et la regression linéaire. Nous testons le modèle avec une Cross Validation égale à 5 et en modifiant le paramètre `fit_intercept` du `LinearRegression`

In [ ]:
model_lr = make_pipeline(preprocessor, LinearRegression())
grid_lr = GridSearchCV(model_lr, param_grid={'linearregression__fit_intercept': [True, False]}, cv=5)
grid_lr.fit(X_train, y_train)
print(grid_lr.score(X_test, y_test))
grid_lr.best_params_

Post: Le modèle a un score de 85,06%, nous obtenons un meilleur score avec en Hyperparamètre: `fit_intercept=True`

Nous pouvons certainement améliorer ce résultat en modifiant les features sélectionnées au bébut. Néanmoins, le résultat semble tout à fait acceptable.